In [1]:
import pandas as pd
import csv
from datetime import date, datetime
import teradatasql
import numpy as np
from dateutil.relativedelta import relativedelta

In [2]:
def process_df(kpi_df, kpi):
    
    nps_df = kpi_df.copy()
    
    cols_dict = {
        'География\n (Total / macro / region)':'geo',
        'География':'geo',
        'Выбор':'area',
        'Оператор':'operator',    
    }
    nps_df.rename(columns=cols_dict, inplace=True)

    for col in nps_df.columns:
        if 'Изменение' in col: nps_df.drop([col], axis=1, inplace=True)
            
    nps_df1 = nps_df.melt(id_vars = ['geo', 'area', 'operator'],
         value_vars=nps_df.columns[3:],
         var_name='quarter',
         value_name='value')
    
    nps_df1.quarter = nps_df1.quarter.apply(lambda x: quarter_to_date(x))
        
    nps_df2 = nps_df1.copy()
    nps_df2 = nps_df2.loc[~nps_df1.geo.str.contains('aggr')]

    nps_df2.area = nps_df2.area.replace(mcrregions_dict)
    nps_df2.operator = nps_df2.operator.replace(operators_dict)
    
    geo_is_total = (nps_df2.geo.str.contains('Total'))
    area_is_not_total_russia_all = (nps_df2.area != 'Total Russia all')
    nps_df2 = nps_df2.loc[~(geo_is_total & area_is_not_total_russia_all)]

    nps_df3 = nps_df2.copy()
    # nps_df3.region = nps_df3.loc[nps_df3.geo.str.contains('Region')].area
    nps_df3['region'] = nps_df3.apply(lambda row: row.area if 'Region' in row.geo else None, axis = 1)
    nps_df3['macroregion'] = nps_df3.apply(lambda row: row.area if 'Macro' in row.geo else None, axis = 1)
    nps_df3['total'] = nps_df3.apply(lambda row: row.area if 'Total' in row.geo else None, axis = 1)
    nps_df3['kpi'] = kpi
    
    nps_df4 = nps_df3.copy()
    for months_count in range(1,3):
        temp_df = nps_df3.copy()
        temp_df.quarter = temp_df.quarter + relativedelta(months=months_count)
        nps_df4 = pd.concat([nps_df4,temp_df])
    
    return nps_df4

In [3]:
kpis = ['NPS','VFM','Consideration']
filename = '4Q2020 Экспресс-мониторинг показателей трекинга.xlsx'
df = {}
for kpi in kpis:
    df[kpi] = pd.read_excel(filename, sheet_name = kpi)
    print(f'{kpi}: ', df[kpi].shape)

NPS:  (336, 29)
VFM:  (439, 29)
Consideration:  (439, 29)


In [4]:
nps_df = df['NPS']
nps_df.head()

,География\n (Total / macro / region),Выбор,Оператор,Q1'15,Q2'15,Q3'15,Q4'15,Q1'16,Q2'16,Q3'16,...,Q1'19,Q2'19,Q3'19,Q4'19,Q1'20,Q2'20,Q3'20,Q4'20,Изменение показателя за квартал (Q4'20 vs. Q3'20),Изменение показателя за год (Q4'20 vs. Q4'19)
0,Macro New,Center,МТС,0.286062,0.294185,0.237860,0.230054,0.242387,0.217836,0.223553,...,0.316822,0.284806,0.344351,0.308960,0.329923,0.356078,0.285102,0.306700,0.021599,-0.002260
1,Macro New,Center,Билайн,0.279883,0.264690,0.220559,0.266975,0.266412,0.270086,0.252601,...,0.142127,0.071441,0.058233,0.088900,0.091601,0.154196,0.146620,0.194914,0.048294,0.106014
2,Macro New,Center,Мегафон,0.303013,0.299744,0.206763,0.193851,0.247607,0.193814,0.198146,...,0.157333,0.152692,0.078542,0.138632,0.136218,0.098666,0.174954,0.214940,0.039986,0.076309
3,Macro New,Center,Tele2,0.547177,0.512726,0.464670,0.496941,0.492511,0.447726,0.455084,...,0.438145,0.415255,0.431809,0.452330,0.405199,0.393193,0.416976,0.452682,0.035706,0.000352
4,Macro New,Chernozem,МТС,0.274883,0.306961,0.260267,0.288708,0.278275,0.277323,0.247762,...,0.358178,0.329040,0.355466,0.330503,0.335203,0.286920,0.335350,0.327888,-0.007461,-0.002615


In [5]:
cols_dict = {
    'География\n (Total / macro / region)':'geo',
    'География':'geo',
    'Выбор':'area',
    'Оператор':'operator',    
}
nps_df.rename(columns=cols_dict, inplace=True)

for col in nps_df.columns:
    if 'Изменение' in col: nps_df.drop([col], axis=1, inplace=True)
        
nps_df.columns

Index(['geo', 'area', 'operator', 'Q1'15', 'Q2'15', 'Q3'15', 'Q4'15', 'Q1'16',
       'Q2'16', 'Q3'16', 'Q4'16', 'Q1'17', 'Q2'17', 'Q3'17', 'Q4'17', 'Q1'18',
       'Q2'18', 'Q3'18', 'Q4'18', 'Q1'19', 'Q2'19', 'Q3'19', 'Q4'19', 'Q1'20',
       'Q2'20', 'Q3'20', 'Q4'20'],
      dtype='object')

In [6]:
nps_df1 = nps_df.melt(id_vars = ['geo', 'area', 'operator'],
         value_vars=nps_df.columns[3:],
         var_name='quarter',
         value_name='value')
nps_df1.head()

,geo,area,operator,quarter,value
0,Macro New,Center,МТС,Q1'15,0.286062
1,Macro New,Center,Билайн,Q1'15,0.279883
2,Macro New,Center,Мегафон,Q1'15,0.303013
3,Macro New,Center,Tele2,Q1'15,0.547177
4,Macro New,Chernozem,МТС,Q1'15,0.274883


In [7]:
def quarter_to_date(q_string):
    q_m = {'Q1':1,'Q2':4,'Q3':7,'Q4':10}
    q_str, y_str = q_string.split("'")   
    return date(2000 + int(y_str), int(q_m[q_str]), 1)

nps_df1.quarter = nps_df1.quarter.apply(lambda x: quarter_to_date(x))
nps_df1.head()

,geo,area,operator,quarter,value
0,Macro New,Center,МТС,2015-01-01,0.286062
1,Macro New,Center,Билайн,2015-01-01,0.279883
2,Macro New,Center,Мегафон,2015-01-01,0.303013
3,Macro New,Center,Tele2,2015-01-01,0.547177
4,Macro New,Chernozem,МТС,2015-01-01,0.274883


In [8]:
nps_df2 = nps_df1.copy()
nps_df2 = nps_df2.loc[~nps_df1.geo.str.contains('aggr')]
regions_old_list = list(nps_df2.loc[nps_df2.geo.str.contains('Region')]['area'].unique())
with teradatasql.connect() as session:
    query = '''
        sel            
            distinct r.region_name as "region_name"
        from PRD2_DIC_V.BRANCH b
        inner join PRD2_DIC_V.REGION r
            on b.region_id=r.region_id
        where product_cluster_name is not null
            and branch_id is not null
            and b.branch_name not like '%CDMA%'
            and b.branch_name not like '%MVNO%'
            and b.branch_name not like '%LTE450%'
            and b.product_cluster_name<>'Deferred'

    '''
    regions_df = pd.read_sql(query, session)
    regions_list = regions_df.region_name.to_list()
unknown_regions = []
for region_old in regions_old_list:
    if region_old not in regions_list:
        unknown_regions.append(region_old)
unknown_regions #Тыва почему-то есть в маркетинговых показателях, пускай она будет

['Тыва']

In [9]:
mcrregions_old_list = list(nps_df2.loc[nps_df2.geo.str.contains('Macro')]['area'].unique())
with teradatasql.connect() as session:
    query = '''
        sel            
            distinct b.macro_cc_name as "mcrregion_name"
        from PRD2_DIC_V.BRANCH b
        inner join PRD2_DIC_V.REGION r
            on b.region_id=r.region_id
        where product_cluster_name is not null
            and branch_id is not null
            and b.branch_name not like '%CDMA%'
            and b.branch_name not like '%MVNO%'
            and b.branch_name not like '%LTE450%'
            and b.product_cluster_name<>'Deferred'

    '''
    mcrregions_df = pd.read_sql(query, session)
mcrregions_list = mcrregions_df.mcrregion_name.to_list()

mcrregions_dict = {
'Center':'Центр',
'Chernozem':'Черноземье',
'Baikal & Far East':'Байкал и Дальний Восток',
'North-West':'Северо-Запад',
'Siberia':'Сибирь',
'South':'Юг',
'Volga':'Волга',
'Ural':'Урал',
'Moscow':'Москва и область',
}

nps_df2.area = nps_df2.area.replace(mcrregions_dict)
unknown_mcrregions = []
for mcrregion_old in mcrregions_old_list:
    if mcrregion_old not in mcrregions_old_list:
        unknown_mcrregions.append(mcrregion_old)
unknown_mcrregions

[]

In [10]:
unknown_mcrregions = []
for mcrregion_old in mcrregions_old_list:
    if mcrregion_old not in mcrregions_old_list:
        unknown_mcrregions.append(mcrregion_old)
unknown_mcrregions

[]

In [11]:
operators_dict = {
'Tele2':'Tele2',
'Yota':'YOTA',
'МТС':'MTS',
'Билайн':'BEE',
'Мегафон':'MGF',
'Мотив':'Мотив',
}
nps_df2.operator = nps_df2.operator.replace(operators_dict)
nps_df2.head()

,geo,area,operator,quarter,value
0,Macro New,Центр,MTS,2015-01-01,0.286062
1,Macro New,Центр,BEE,2015-01-01,0.279883
2,Macro New,Центр,MGF,2015-01-01,0.303013
3,Macro New,Центр,Tele2,2015-01-01,0.547177
4,Macro New,Черноземье,MTS,2015-01-01,0.274883


In [12]:
geo_is_total = (nps_df2.geo.str.contains('Total'))
area_is_not_total_russia_all = (nps_df2.area != 'Total Russia all')
nps_df2 = nps_df2.loc[~(geo_is_total & area_is_not_total_russia_all)]
nps_df2.loc[nps_df2.geo.str.contains('Total')]['area'].unique()

array(['Total Russia all'], dtype=object)

In [13]:
# nps_df2.value = nps_df2.value.apply(lambda x: round(float(x),4))
# nps_df2.head()

In [14]:
nps_df3 = nps_df2.copy()
# nps_df3.region = nps_df3.loc[nps_df3.geo.str.contains('Region')].area
nps_df3['region'] = nps_df3.apply(lambda row: row.area if 'Region' in row.geo else None, axis = 1)
nps_df3['macroregion'] = nps_df3.apply(lambda row: row.area if 'Macro' in row.geo else None, axis = 1)
nps_df3['total'] = nps_df3.apply(lambda row: row.area if 'Total' in row.geo else None, axis = 1)
nps_df3['kpi'] = 'NPS'
nps_df3.head()

,geo,area,operator,quarter,value,region,macroregion,total,kpi
0,Macro New,Центр,MTS,2015-01-01,0.286062,None,Центр,None,NPS
1,Macro New,Центр,BEE,2015-01-01,0.279883,None,Центр,None,NPS
2,Macro New,Центр,MGF,2015-01-01,0.303013,None,Центр,None,NPS
3,Macro New,Центр,Tele2,2015-01-01,0.547177,None,Центр,None,NPS
4,Macro New,Черноземье,MTS,2015-01-01,0.274883,None,Черноземье,None,NPS


In [15]:
nps_df4 = nps_df3.copy()
print(nps_df4.shape)
for months_count in range(1,3):
    temp_df = nps_df3.copy()
    temp_df.quarter = temp_df.quarter + relativedelta(months=months_count)
    nps_df4 = pd.concat([nps_df4,temp_df])
print(nps_df4.shape)

(6720, 9)
(20160, 9)


In [34]:
import os
path = os.getcwd() + '\\'
for file in enumerate(os.listdir(path)):
    print(file)
file_index = int(input('Какой номер нужен? '))
shortname = None
for i, file in enumerate(os.listdir(path)):
    if i == file_index:
        print(f'Выбран {file}')
        shortname = file
        filename = path + shortname
        break
if not shortname:
    print('Файл не выбран')

(0, '.ipynb_checkpoints')
(1, '2Q2020 Экспресс-мониторинг показателей трекинга.xlsx')
(2, '3Q2020 Экспресс-мониторинг показателей трекинга.xlsx')
(3, '4Q2020 Экспресс-мониторинг показателей трекинга.xlsx')
(4, 'process and insert tracking parameters.ipynb')
Какой номер нужен? 3
Выбран 4Q2020 Экспресс-мониторинг показателей трекинга.xlsx


In [35]:
kpis = ['NPS','VFM','Consideration']
union_df = pd.DataFrame()

for kpi in kpis:
    kpi_df = pd.read_excel(filename, sheet_name = kpi)
    print(f'{kpi}: ', kpi_df.shape)
    processed_df = process_df(kpi_df, kpi)
    union_df = pd.concat([union_df,processed_df]).drop_duplicates()
print(f'UNION: ', union_df.shape)
union_df.head()

NPS:  (336, 29)
VFM:  (439, 29)
Consideration:  (439, 29)
UNION:  (73296, 9)


,geo,area,operator,quarter,value,region,macroregion,total,kpi
0,Macro New,Центр,MTS,2015-01-01,0.286062,None,Центр,None,NPS
1,Macro New,Центр,BEE,2015-01-01,0.279883,None,Центр,None,NPS
2,Macro New,Центр,MGF,2015-01-01,0.303013,None,Центр,None,NPS
3,Macro New,Центр,Tele2,2015-01-01,0.547177,None,Центр,None,NPS
4,Macro New,Черноземье,MTS,2015-01-01,0.274883,None,Черноземье,None,NPS


In [20]:
cols = ['quarter', 'region', 'macroregion',
        'total', 'kpi', 'operator', 'value']
union_df = union_df.reindex(columns = cols,  fill_value = np.NaN).where((pd.notnull(union_df)), None)
union_df.head()

,quarter,region,macroregion,total,kpi,operator,value
0,2015-01-01,None,Центр,None,NPS,MTS,0.286062
1,2015-01-01,None,Центр,None,NPS,BEE,0.279883
2,2015-01-01,None,Центр,None,NPS,MGF,0.303013
3,2015-01-01,None,Центр,None,NPS,Tele2,0.547177
4,2015-01-01,None,Черноземье,None,NPS,MTS,0.274883


In [21]:
# чистим TRACKING_PARAMETERS и заполняем данными
df = union_df.copy()

with teradatasql.connect() as con:
    with con.cursor() as cur:
 
        print('deleting from uat_product.TRACKING_PARAMETERS...')
        cur.execute('''
            delete from uat_product.TRACKING_PARAMETERS;
        ''')
        print(f'{cur.rowcount} rows deleted from uat_product.TRACKING_PARAMETERS')
        
        batchsize = 100000
        print('inserting into uat_product.TRACKING_PARAMETERS...')
        for num in range(0, len(df), batchsize):
            cur.executemany(f'''
             INSERT into uat_product.TRACKING_PARAMETERS ({','.join('?'*len(df.columns))})
            ''',
                [tuple(row) for row in df.iloc[num:num+batchsize,:].itertuples(index=False)]
                )
        print(f'{len(df)} rows inserted into uat_product.TRACKING_PARAMETERS.')
        
        print('collecting statistics...')
        cur.execute('''
        COLLECT STATISTICS
            COLUMN(REPORT_QUARTER)
            ,COLUMN(REGION)
            ,COLUMN(MACROREGION)
            ,COLUMN(TOTAL)
            ,COLUMN(KPI)
            ,COLUMN(MOBILE_OPERATOR)
            ON UAT_PRODUCT.TRACKING_PARAMETERS;
                ''')
        print(f'{cur.rowcount} columns done.')

deleting from uat_product.TRACKING_PARAMETERS...
69207 rows deleted from uat_product.TRACKING_PARAMETERS
inserting into uat_product.TRACKING_PARAMETERS...
73296 rows inserted into uat_product.TRACKING_PARAMETERS.
collecting statistics...
7 columns done.
